# 5. Auditoría de seguridad y criptografía

Este notebook demuestra y valida las capas de seguridad implementadas en la aplicación. Simulararemos ataques y verificaremos la integridad de los mecanismos de protección de datos.

### Tecnologías implementadas:
1.  **Hashing de password:** Algoritmo **Scrypt** (Protección contra ataques de fuerza bruta y Rainbow Tables).
2.  **Cifrado de datos (PII):** Cifrado Simétrico **AES-128 (Fernet)** para datos sensibles en BBDD.
3.  **Firma digital (PKI):** Infraestructura de Clave Pública (**RSA-2048**) para garantizar la autenticidad de los informes PDF.
4.  **Tokens Seguros:** Generación de entropía segura para enlaces temporales.

## 5.1. Configuración inicial

Definimos la importación de librerías necesarias de bbdd (SQLite3) y de criptografía, así como las rutas relativas necesarias para testear la ciberseguridad de nuestro MVP.

In [ ]:
import os
import sys
import sqlite3
import pandas as pd

# Ajustamos las rutas para importar módulos del src
current_dir = os.getcwd()
src_path = os.path.join(os.path.dirname(current_dir), 'src')
sys.path.append(src_path)

# Importamos nuestro motor de seguridad
from crypto_manager import (
    hashear_password, 
    verificar_password, 
    cifrar_pii, 
    descifrar_pii, 
    firmar_digitalmente,
    generar_token_seguro
)

print(f"[*] Módulos de seguridad cargados desde: {src_path}")

🔑 Nueva clave maestra de cifrado generada.
🔐 Generando PKI Corporativa para GeSAI...
✅ Identidad Digital creada en /src/keys/
✅ Módulos de seguridad cargados desde: c:\Users\barco\OneDrive\Documentos\GitHub\GeSAI-AB_Data_Challenge\src


## 5.2. Protección de credenciales (Hashing)
Intentaremos "robar" una contraseña. Veremos la diferencia entre guardar texto plano y guardar un Hash con Salt.

In [ ]:
password_real = "Admin1234.!"

# 1. Generar Hash
hash_seguro = hashear_password(password_real)

print(f"[*] Contraseña Original: '{password_real}'")
print(f"[*] Lo que se guarda en BBDD (Scrypt):")
print(f"   {hash_seguro}")
print("-" * 60)

# 2. Intento de verificación
intento_hacker = "123456"
intento_correcto = "Admin1234.!"

print(f"Intento con '{intento_hacker}': {verificar_password(intento_hacker, hash_seguro)}")
print(f"Intento con '{intento_correcto}':  {verificar_password(intento_correcto, hash_seguro)}")

# 3. Prueba de Salt (Misma contraseña, distinto hash)
hash_seguro_2 = hashear_password(password_real)
print("-" * 60)
print("Prueba de aleatoriedad (Salt):")
print(f"Hash 1: {hash_seguro[:20]}...")
print(f"Hash 2: {hash_seguro_2[:20]}...")
print("[*] Conclusión: Incluso si dos usuarios tienen la misma contraseña, sus hashes son distintos.")

🔒 Contraseña Original: 'Admin1234.!'
🛡️ Lo que se guarda en BBDD (Scrypt):
   f32470c25cd2f2e9b4d46a2944bbd166$734cc04fb5c6a8715936a5ffbd690857fbea4c4065d579524904d0e9a1e48562
------------------------------------------------------------
Intento con '123456': False
Intento con 'Admin1234.!':  True
------------------------------------------------------------
Prueba de aleatoriedad (Salt):
Hash 1: f32470c25cd2f2e9b4d4...
Hash 2: 8faa3c5a16701e61e7f6...
✅ Conclusión: Incluso si dos usuarios tienen la misma contraseña, sus hashes son distintos.


## 5.3. Privacidad de datos (Cifrado AES)
Simulamos ser un atacante que ha conseguido acceso al archivo `gesai.db`. Intentamos leer los nombres de los clientes.

In [ ]:
# Conexión directa a la BBDD (sin pasar por la app)
db_path = os.path.join(os.path.dirname(current_dir), 'gesai.db')
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

print("[*] SIMULACIÓN DE ROBO DE DATOS (SQL RAW):")
print("-" * 60)

try:
    # Leemos la tabla clientes tal cual está en el disco
    df_robado = pd.read_sql("SELECT cliente_id, nombre, email FROM clientes LIMIT 3", conn)
    
    for index, row in df_robado.iterrows():
        print(f"ID: {row['cliente_id']}")
        print(f"Nombre (Cifrado): {row['nombre']}")
        print(f"Email (Cifrado):  {row['email']}")
        print("." * 40)
        
    print("\n[*]Conclusión: Los datos robados son inútiles sin la clave maestra.")
    
except Exception as e:
    print(f"Error leyendo BBDD: {e}")
finally:
    conn.close()

🕵️‍♂️ SIMULACIÓN DE ROBO DE DATOS (SQL RAW):
------------------------------------------------------------
ID: X2JO733NUICX5BL3
Nombre (Cifrado): gAAAAABpMvDlGthpQJWbPeAya58nyKXEpA2k3gJQfNdaO6tP1xOAqZhWVh47faWUSXkvbuky4fF27oNSgrplrqBaiYNSaHydiw==
Email (Cifrado):  None
........................................
ID: TOUO4OAJPCNOO6UN
Nombre (Cifrado): gAAAAABpMvDlxDSk6aHHcLK3l2eKAmJKhcsbRj-zXhCQJBz7eSClu07idpmeKEZbDkrGnoCijfwJOkZtBo1RiIzkTSR6sgVG6g==
Email (Cifrado):  gAAAAABpMvDlDncOSLdVd-JmKBjA5AYJxBq2RSohgRDQ4sF7iIK1WGN5HzS4VDGoPbrpO7PNMo_ksdcG2V4X5WBsUQsZy_IW7A==
........................................
ID: UPBXZAO3MXBDYZDH
Nombre (Cifrado): gAAAAABpMvDlKBq-nH-7ef59mLS0UEgB5llOKNMgunMHZd3qnElByt0FiGQ_ZnbO8dz38_diCr42KySBXgVQTGnorhJ6MwwxaR-V5V5s-BJSr1nYhPBL8pc=
Email (Cifrado):  gAAAAABpMvDlRGQlvlHR_CYwsGKDIfOFuFsp10vmdBn-SC7eSIMZrhTpvyRqRhtlHRR-a874Prl2XXT84fTNIM7TMRQqMFS4MA==
........................................

✅ Conclusión: Los datos robados son inútiles sin la clave maestra.


In [ ]:
# Ahora actuamos como la aplicación legítima usando la clave maestra
print("[*] ACCESO LEGÍTIMO (APP):")
print("-" * 60)

# Tomamos el primer nombre cifrado del ejemplo anterior
nombre_cifrado = df_robado.iloc[0]['nombre']
email_cifrado = df_robado.iloc[0]['email']

nombre_real = descifrar_pii(nombre_cifrado)
email_real = descifrar_pii(email_cifrado)

print(f"Cifrado:   {nombre_cifrado[:30]}...")
print(f"Descifrado: {nombre_real}")
print(f"Email:      {email_real}")

🔓 ACCESO LEGÍTIMO (APP):
------------------------------------------------------------
Cifrado:   gAAAAABpMvDlGthpQJWbPeAya58nyK...
Descifrado: José del Sales
Email:      None


## 5.4. Integridad Documental (Firma RSA)
GeSAI actúa como una Autoridad de Certificación (CA). 

Firmamos digitalmente un resumen del informe para garantizar que no ha sido alterado.

In [ ]:
# Simulamos los datos de un informe
incidencia_id = 101
fecha = "05/12/2025"
cliente_id = "XJ99"
estado = "FUGA GRAVE"

# 1. Creamos el "Fingerprint" (Resumen único del documento)
documento_fingerprint = f"DOC:INFORME|ID:{incidencia_id}|CLI:{cliente_id}|DATE:{fecha}|STATUS:{estado}"
print(f"[*] Huella del documento: {documento_fingerprint}")

# 2. Firmamos con la Clave Privada (Solo el servidor la tiene)
firma = firmar_digitalmente(documento_fingerprint.encode('utf-8'))

print(f"\n[*] FIRMA DIGITAL GENERADA (RSA-2048):")
print(f"{firma[:100]}...") # Mostramos solo el principio
print(f"(Longitud total: {len(firma)} caracteres hexadecimales)")

print("\n[*]Esta firma es matemáticamente imposible de generar sin la Clave Privada de GeSAI.")

📄 Huella del documento: DOC:INFORME|ID:101|CLI:XJ99|DATE:05/12/2025|STATUS:FUGA GRAVE

✍️ FIRMA DIGITAL GENERADA (RSA-2048):
879b6aaa083d7ffa627f315d2a379d5b5b0ff7d724efd2e2229c8df9f95ca85c7c8cc64feb6baf0bdd21b076b7625325fda1...
(Longitud total: 512 caracteres hexadecimales)

✅ Esta firma es matemáticamente imposible de generar sin la Clave Privada de GeSAI.
